In [ ]:
import pandas as pd
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.visualization import plot_histogram
from math import pi
from qiskit import Aer
import time
# Load your new CSV file
file_path = 'chicago_crime.csv'  # Replace 'new_file.csv' with the actual file name
df = pd.read_csv(file_path)

# Define the oracle based on the search key
def mark_state(circuit, qr_target, qr_auxiliary, n, search_key):
    for qubit, bit in enumerate(search_key):
        if bit == '1' and qubit < n:
            circuit.x(qr_target[qubit])
    for qubit in range(min(n, len(qr_auxiliary))):
        circuit.cx(qr_target[qubit], qr_auxiliary[qubit])
    for qubit, bit in enumerate(search_key):
        if bit == '1' and qubit < n:
            circuit.x(qr_target[qubit])

# Define Grover's diffusion operator
def diffusion(circuit, qr, n):
    circuit.h(qr)
    circuit.x(qr)
    circuit.h(qr[n-1])
    circuit.mcx(qr[:-1], qr[n-1])  # multi-controlled-x
    circuit.h(qr)
    circuit.x(qr)
    circuit.h(qr[n-1])

# Grover's search algorithm
def grover_search(search_column, search_value):
    start_time = time.time()
    n = 10  # Reduced the number of qubits for simplicity

    # Print available column names for verification
    print("Available column names:", df.columns)

    # Check if the specified column exists (case-insensitive)
    search_column_lower = search_column.lower()
    matching_columns = [col for col in df.columns if col.lower() == search_column_lower]

    if not matching_columns:
        print(f"Column '{search_column}' not found in the DataFrame.")
        return

    # Convert the entire column to lowercase for case-insensitive search
    df_lower = df.applymap(lambda x: x.lower() if type(x) == str else x)

    # Find the indices corresponding to the search key
    search_value_str = str(search_value)
    indices = df_lower[df_lower[matching_columns[0]] == search_value_str.lower()].index

    # Debugging: Print indices and search_value_binary
    print("Indices:", indices)

    matching_rows = []

    for index in indices:
        # Debugging: Print search_value_binary before using it in the quantum circuit
        search_value_binary = format(index, '010b')
        
        qr_target = QuantumRegister(n, 'target')  # Quantum register for the target
        qr_auxiliary = QuantumRegister(n, 'auxiliary')  # Quantum register for the auxiliary
        cr = ClassicalRegister(n, 'result')  # Classical register to measure the result
        qc = QuantumCircuit(qr_target, qr_auxiliary, cr)

        # Apply Hadamard gates to all qubits
        qc.h(qr_target)

        iterations = int((pi/4) * (2**((n-1)/2)))  # Number of Grover iterations

        for _ in range(iterations):
            mark_state(qc, qr_target, qr_auxiliary, n, search_value_binary)
            diffusion(qc, qr_target, n)

        # Measure qubits
        qc.measure(qr_target, cr)

        # Execute the quantum circuit
        backend = Aer.get_backend('qasm_simulator')
        job = backend.run(transpile(qc, backend, optimization_level=0))  # Set optimization level to 0
        result = job.result()

        # Get the measurement outcome
        counts = result.get_counts()
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Runtime: {elapsed_time} ms")

        # Check if the search key is in the counts
        if search_value_binary in counts:
            # Print the matching row directly
            print(f"\nRow containing '{search_value_str}' in column '{search_column}':\n{df.iloc[index]}")


# Example usage:
search_column = input("Enter the column name to search: ")
search_value = input(f"Enter the value in column '{search_column}' to search: ")
grover_search(search_column, search_value)

Enter the column name to search: PRIMARY TYPE
Enter the value in column 'PRIMARY TYPE' to search: BATTERY
Available column names: Index(['case_number', 'id', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')


C:\Users\91981\AppData\Local\Temp\ipykernel_23264\523299121.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_lower = df.applymap(lambda x: x.lower() if type(x) == str else x)


Indices: Index([     1,      3,      7,      8,     11,     17,     21,     24,     25,
           29,
       ...
       212168, 212169, 212182, 212191, 212193, 212201, 212221, 212226, 212228,
       212238],
      dtype='int64', length=41514)
Runtime: 5.229853391647339 ms

Row containing 'BATTERY' in column 'PRIMARY TYPE':
case_number                                  12031001
id                                           JD209965
Date                                   4/16/2020 5:00
Block                                 005XX W 32ND ST
IUCR                                              460
Primary Type                                  BATTERY
Description                                    SIMPLE
Location Description                        APARTMENT
Arrest                                           True
Domestic                                        False
Beat                                              915
District                                            9
Ward                      

Runtime: 12.752997636795044 ms
Runtime: 13.595247983932495 ms
Runtime: 14.44932246208191 ms

Row containing 'BATTERY' in column 'PRIMARY TYPE':
case_number                                  12122483
id                                           JD316923
Date                                  7/31/2020 23:15
Block                         021XX N SPRINGFIELD AVE
IUCR                                              486
Primary Type                                  BATTERY
Description                   DOMESTIC BATTERY SIMPLE
Location Description                        APARTMENT
Arrest                                          False
Domestic                                         True
Beat                                             2525
District                                           25
Ward                                             26.0
Community Area                                   22.0
FBI Code                                          08B
X Coordinate                                11

Runtime: 26.294708490371704 ms

Row containing 'BATTERY' in column 'PRIMARY TYPE':
case_number                                  12196042
id                                           JD402456
Date                                 10/17/2020 14:30
Block                             051XX S PRAIRIE AVE
IUCR                                              486
Primary Type                                  BATTERY
Description                   DOMESTIC BATTERY SIMPLE
Location Description               RESIDENCE - GARAGE
Arrest                                          False
Domestic                                         True
Beat                                              231
District                                            2
Ward                                              3.0
Community Area                                   40.0
FBI Code                                          08B
X Coordinate                                1178914.0
Y Coordinate                                1870938.0

...

Runtime: 890.9173669815063 ms
Runtime: 891.7721309661865 ms
Runtime: 892.6036047935486 ms
Runtime: 893.4465351104736 ms
Runtime: 894.2762784957886 ms
Runtime: 895.10808801651 ms
Runtime: 895.9751422405243 ms
Runtime: 896.821483373642 ms
Runtime: 897.6771194934845 ms
Runtime: 898.50829911232 ms
Runtime: 899.3402843475342 ms
Runtime: 900.2005052566528 ms
Runtime: 901.0497915744781 ms
Runtime: 902.0663945674896 ms
Runtime: 903.0278437137604 ms
Runtime: 903.9906666278839 ms
Runtime: 904.8663153648376 ms
Runtime: 905.7250723838806 ms
Runtime: 906.555915594101 ms
Runtime: 907.4104874134064 ms
Runtime: 908.2197580337524 ms
Runtime: 909.0471000671387 ms
Runtime: 909.8832054138184 ms
Runtime: 910.7086684703827 ms
Runtime: 911.5418066978455 ms
Runtime: 912.3644683361053 ms
Runtime: 913.1988179683685 ms
Runtime: 914.0311944484711 ms
Runtime: 914.8640305995941 ms
Runtime: 915.8290345668793 ms
Runtime: 916.670125246048 ms
Runtime: 917.5006628036499 ms
Runtime: 918.3373210430145 ms
Runtime: 919.1493

Runtime: 1121.35795378685 ms
Runtime: 1122.1819806098938 ms
Runtime: 1123.130981206894 ms
Runtime: 1123.9541475772858 ms
Runtime: 1124.789793729782 ms
Runtime: 1125.6058082580566 ms
Runtime: 1126.4301385879517 ms
Runtime: 1127.2291588783264 ms
Runtime: 1128.058052778244 ms
Runtime: 1128.8747704029083 ms
Runtime: 1129.718951702118 ms
Runtime: 1130.5154778957367 ms
Runtime: 1131.4394810199738 ms
Runtime: 1132.3646290302277 ms
Runtime: 1133.245284318924 ms
Runtime: 1134.1762871742249 ms
Runtime: 1135.054289340973 ms
Runtime: 1135.971173286438 ms
Runtime: 1137.0111134052277 ms
Runtime: 1137.8626880645752 ms
Runtime: 1138.7309052944183 ms
Runtime: 1139.547284603119 ms
Runtime: 1140.3755412101746 ms
Runtime: 1141.2984352111816 ms
Runtime: 1142.1431498527527 ms
Runtime: 1142.9801850318909 ms
Runtime: 1143.8398411273956 ms
Runtime: 1144.6810250282288 ms
Runtime: 1145.5421993732452 ms
Runtime: 1146.3807446956635 ms
Runtime: 1147.215591430664 ms
Runtime: 1148.0718257427216 ms
Runtime: 1148.92161

Runtime: 1351.7897040843964 ms
Runtime: 1352.658822774887 ms
Runtime: 1353.5111479759216 ms
Runtime: 1354.3827958106995 ms
Runtime: 1355.2254886627197 ms
Runtime: 1356.107577085495 ms
Runtime: 1356.9508674144745 ms
Runtime: 1357.8074204921722 ms
Runtime: 1358.6433551311493 ms
Runtime: 1359.6396796703339 ms
Runtime: 1360.4634294509888 ms
Runtime: 1361.3152668476105 ms
Runtime: 1362.1551988124847 ms
Runtime: 1363.008977174759 ms
Runtime: 1363.848270893097 ms
Runtime: 1364.6823284626007 ms
Runtime: 1365.5288136005402 ms
Runtime: 1366.3575599193573 ms
Runtime: 1367.2276990413666 ms
Runtime: 1368.066205739975 ms
Runtime: 1368.9074041843414 ms
Runtime: 1369.7582178115845 ms
Runtime: 1370.596999168396 ms
Runtime: 1371.5512053966522 ms
Runtime: 1372.3933284282684 ms
Runtime: 1373.2442574501038 ms
Runtime: 1374.0664789676666 ms
Runtime: 1374.9243457317352 ms
Runtime: 1375.7784473896027 ms
Runtime: 1376.6324293613434 ms
Runtime: 1377.4588401317596 ms
Runtime: 1378.3330342769623 ms
Runtime: 1379.

Runtime: 1582.4859683513641 ms
Runtime: 1583.3275876045227 ms
Runtime: 1584.222985982895 ms
Runtime: 1585.0378849506378 ms
Runtime: 1585.8596227169037 ms
Runtime: 1586.6959536075592 ms
Runtime: 1587.6263461112976 ms
Runtime: 1588.4363205432892 ms
Runtime: 1589.3059015274048 ms
Runtime: 1590.1278984546661 ms
Runtime: 1590.9566059112549 ms
Runtime: 1591.823900938034 ms
Runtime: 1592.6631090641022 ms
Runtime: 1593.4835948944092 ms
Runtime: 1594.3160285949707 ms
Runtime: 1595.1290094852448 ms
Runtime: 1595.9437544345856 ms
Runtime: 1596.787141084671 ms
Runtime: 1597.6367938518524 ms
Runtime: 1598.467924118042 ms
Runtime: 1599.3430485725403 ms
Runtime: 1600.1729905605316 ms
Runtime: 1601.1111690998077 ms
Runtime: 1601.9549028873444 ms
Runtime: 1602.7815117835999 ms
Runtime: 1603.6220548152924 ms
Runtime: 1604.4759442806244 ms
Runtime: 1605.321043252945 ms
Runtime: 1606.169585466385 ms
Runtime: 1606.9910118579865 ms
Runtime: 1607.8263628482819 ms
Runtime: 1608.644284248352 ms
Runtime: 1609.4

Runtime: 1813.2834341526031 ms
Runtime: 1814.1204617023468 ms
Runtime: 1814.9789128303528 ms
Runtime: 1815.86860704422 ms
Runtime: 1816.8496487140656 ms
Runtime: 1817.8011627197266 ms
Runtime: 1818.7868208885193 ms
Runtime: 1819.7536797523499 ms
Runtime: 1820.624641418457 ms
Runtime: 1821.5750963687897 ms
Runtime: 1822.5220174789429 ms
Runtime: 1823.2811431884766 ms
Runtime: 1824.0975337028503 ms
Runtime: 1824.9415273666382 ms
Runtime: 1825.8168261051178 ms
Runtime: 1826.666998386383 ms
Runtime: 1827.4516386985779 ms
Runtime: 1828.2754600048065 ms
Runtime: 1829.1775600910187 ms
Runtime: 1830.0735902786255 ms
Runtime: 1831.063503742218 ms
Runtime: 1831.9233448505402 ms
Runtime: 1832.8129818439484 ms
Runtime: 1833.758902311325 ms
Runtime: 1834.6385962963104 ms
Runtime: 1835.5204515457153 ms
Runtime: 1836.3670823574066 ms
Runtime: 1837.2197947502136 ms
Runtime: 1838.0904953479767 ms
Runtime: 1838.9666543006897 ms
Runtime: 1839.7818942070007 ms
Runtime: 1840.6280980110168 ms
Runtime: 1841.